# Working with files

In [2]:
!cat hello.py
print('Hello, world!')

def print_hello():
    """Just print hello"""
    print('Hello, world!')

def print2():
    pass

def print3():
    pass
    Hello, world!


In [8]:
!cat use.py

import hello
from hello import print_hello
import hello as h
import math as m

# similar to math.pi
hello.print_hello()
print_hello()
h.print_hello()
print(m.pi)

In [7]:
!python use.py

Hello, world!
Hello, world!
Hello, world!
3.14159265359


In [9]:
!cat add.py

"""Add one to command line input"""

import sys

def add_one(x):
    return x + 1

if __name__ == '__main__':
    number = int(sys.argv[1])
    result = add_one(number)
    print(result)


In [11]:
!python add.py

Traceback (most recent call last):
  File "add.py", line 9, in <module>
    number = int(sys.argv[1])
IndexError: list index out of range


In [12]:
!python add.py 2

3


# Reading files in custom format

In [14]:
!cat custom.txt

123 = bread
134 = rolls
212 = milk
213 = butter


In [38]:
# open a file for reading and iterate each line
with open('./custom.txt') as f:
    for line in f:
        # similar to line = "123 = bread\n"
        # assign to multiple variables (tuple unpacking)
        id, name = line.strip().split(' = ')
        print(id, name)

123 bread
134 rolls
212 milk
213 butter


## JSON files

In [16]:
!cat original.json

{
    "123": "bread",
    "134": "rolls",
    "212": "milk",
    "213": "butter"
}


In [17]:
"""Read and write JSON files"""

import json

with open('./original.json') as original, \
     open('./output.json', 'w') as output:
    config = json.load(original)
    config["133"] = config["134"]
    del config["134"]
    json.dump(config, output)

In [18]:
!cat output.json

{"123": "bread", "212": "milk", "213": "butter", "133": "rolls"}

# CSV files

In [19]:
!cat original.csv

id;name
123;bread
134;rolls
212;milk
213;butter


In [21]:
!cat csv-simple.py

"""Work with CSV files in normal Python"""

import csv

with open('./original.csv') as file:
    reader_object = csv.reader(file, delimiter=";")
    all_rows = list(reader_object)

with open('./output.csv', 'w', newline='') as output:
    writer_object = csv.writer(output)
    for line in all_rows:
        writer_object.writerow(line)


# for id, name in all_rows:
#     print(id, name)


In [22]:
!cat output.csv

id,name
123,bread
134,rolls
212,milk
213,butter
